In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
inst = (39,269,369)
exp = get_exp(inst,directory_project)
def criter(max_growth,length):
    return(a*length+b*max_growth>=2.5 and max_growth>=50)
RH, BAS, max_speeds, total_growths, widths, lengths, branch_frequ,select_hyph = get_rh_bas(exp,criter)
hyph_anas_tip_hyph = [hyphat for hyphat in exp.hyphaes if len(hyphat.ts)>=2 and hyphat.end.degree(hyphat.ts[-1])>=3 and hyphat.end.degree(hyphat.ts[-2])>=3]
branches = {}
lengths = {}
bd = []
ts = []
rh_ends = [rh.end for rh in RH]
daughters = {}
begins = {}
for rh in RH:
    tp1 = rh.ts[-1]
    t = rh.ts[0]
    pixels, nodes = get_pixel_growth_and_new_children(rh, t, tp1)
    curv_length = np.sum([get_length_um(seg) for seg in pixels])
    branch = []
    daughter_list=[]
    begins[rh] = nodes[1]
    for node in nodes[1:-1]:
        if node not in [hyph.end for hyph in hyph_anas_tip_hyph]:
            node_obj  = node
#             t0 = node_obj.ts()[0]
#             neighbours = node_obj.neighbours(t0)
#             if t0 in rh.ts:
#                 nodes_hyph,edge = rh.get_nodes_within(t0)
#                 for neighbour in neighbours:
#                     if neighbour.label not in nodes_hyph:
#                         daughter_list.append(neighbour)
#                         if neighbour in rh_ends and neighbour!=rh.end:
            branch.append(node)    
    bd.append(len(branch)/curv_length)
    ts.append(get_time(exp,0,t))
    branches[rh] = branch
    daughters[rh] = daughter_list
    lengths[rh] = curv_length

begin = 2020-11-23 15:01:00 
  end = 2020-11-27 19:12:00
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in find

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in double_scalars


failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge
failure in finding closest edge


In [2]:
data = pd.DataFrame(list(zip(ts,bd)),columns = ['time','branchin_dist'])
corrected = data.loc[(data['branchin_dist']>0)]
corrected['real_branchin_dist'] = 1/corrected['branchin_dist']

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
plt.close('all')
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 20})
abcisse = 'time'
ordinate = 'real_branchin_dist'
tab = corrected
baits_sort = tab.sort_values(abcisse)
N=100
moving_av = baits_sort.rolling(N,min_periods=N//2).mean()
moving_std = baits_sort.rolling(N,min_periods=N//2).std()

fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (h)')

ax.set_ylabel('Branching distance (BAS and RH undistinguished) ($\mu m$)',color='red')
ax.set_xlim(0,70)
ax.set_ylim(0,1500)
# ax.set_xlim(-190,190)
# slope, intercept, r_value, p_value, std_err = stats.linregress(densities_sort,np.abs(curvatures_sort))
# ts = list(set(tab[abcisse].values))
# ts.sort()
# averages =  [np.mean(tab.loc[tab[abcisse]==t][ordinate]) for t in ts]
# sts = [np.std(tab.loc[tab[abcisse]==t][ordinate])/np.sqrt(len(tab.loc[tab[abcisse]==t][ordinate])) for t in ts]
# plt.errorbar(ts,averages,yerr=sts, linestyle="None",capthick = 1)
# ax.plot(ts,averages)
ax.scatter(tab[abcisse],tab[ordinate],alpha=0.3,color='red')
ax.plot(moving_av[abcisse],moving_av[ordinate],color='red',label = 'moving average')
ax.plot(moving_av[abcisse],(moving_av[ordinate]+moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.plot(moving_av[abcisse],(moving_av[ordinate]-moving_std[ordinate]/np.sqrt(N)),color='red',label = 'std',linestyle = 'dotted')
ax.spines['right'].set_color('red')
# ax.set_title((inst,np.mean(corrected['real_branchin_dist'])))
ax.tick_params(axis='y', colors='red')  #setting up Y-axis tick color to black

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:1377: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x[:, None]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:237: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x = x[:, np.newaxis]
/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/axes/_base.py:239: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  y = y[:, np.newaxis]


In [4]:
np.mean(corrected['real_branchin_dist'])

790.2496200135315

In [4]:
exp.plot_raw(-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …